# DESP User Workflow Processing Services

## Overview

This document serves as a quick reference guide of how to access **DESP User Workflow Processing** environment APIs, using Python, from the User Workflow Development environment. This guide will show how to discover, launch and monitor processing services.

### Steps
1. Preparation
2. Discover a Processing Service
3. Launch a Processing Service

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each HTTP request used in these examples.


> IMPORTANT NOTE: Please ensure to provide a valid username and token pair.

In [ ]:
import requests
import base64
import json
import tempfile
import zipfile
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import getpass

In [ ]:
username = getpass.getpass()

In [ ]:
token = getpass.getpass()

In [ ]:
credentials = username + ':' + token

authorization = "Basic " + base64.b64encode(bytes(credentials, 'utf-8')).decode('utf-8')
headers = {'Authorization':authorization,'Content-Type':'application/hal+json;charset=UTF-8'}
base_url = "https://userwkfl.dev.desp.space"

## Discover a Processing Service
This step is a straightforward one. By polling the `services` API we get a list of available services and parameters which can be used to execute a Processing Service.

In [ ]:
url = base_url + "/secure/api/v2.0/services?page=0&size=10"
run_request=requests.get(url,headers=headers, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

## Launch a Processing Service

### Create a Configuration

A Job Configuration is a config file that defines what the job should do. This request creates a new job config which is needed for the following section.

The crucial aspects of a job config are:
- ***service***: the `service_id` instructs the job about which service is to be run
- ***inputs***: on which images/files said service is to be executed
- ***collection***: usally for many services this represents the output collection in which the results will be found
Be aware as these parameters must match with the ones that the service expects.

In [ ]:
service_id = 20 # TestProcessor
collection_id = 'eopaasbb235018ba8342dda1df91a5c3678395' # test-jupytr-file
s2 = "Insula is"
s3 = "great!"

output={'result':collection_id}
job_config={
    'service': f"{base_url}secure/api/v2.0/services/{service_id}",
    'inputs':{ # inputs must match the fields required by the service
        's2_input': [s2],
        's3_input': [s3],
        'collection': [json.dumps(output)]
    },
    'parent': None
}

url = base_url + "/secure/api/v2.0/jobConfigs/"
run_request = requests.post(url,headers=headers, data=json.dumps(job_config), verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

We can start a processing service by getting the `job_config_id` from the previous response and launch it through the request you see in the following section:

In [ ]:
jobconfig_id=run_request_dict['id']
url=f"{base_url}/secure/api/v2.0/jobConfigs/{jobconfig_id}/launch"
run_request = requests.post(url, headers=headers, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

To receive updates on the advancement of the job, we're simply getting the `job_id` from the previous response, and asking the API about it.

Feel free to run this section a couple of times to see the status changes until the job is completed.

In [ ]:
job_id = run_request_dict['content']['id']
url_status = f"{base_url}/secure/api/v2.0/jobs/{job_id}"
status = requests.get(url_status, headers=headers)
url_status_dict = json.loads(status.text)